# Making mean zeta with bathymetry

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from netCDF4 import Dataset

import numpy as np
import Nio,Ngl
import os,sys
import datetime



In [2]:
  path_to_output = "/scratch/rhe/GoM/Data-Assimilation/jan2009b/"
  latbound_s  = 24 #21
  latbound_n  = 28 #29
  lonbound_w  = -89 #-91
  lonbound_e  = -84 #-82
  depths      = 0 #np.linspace(-1, -2000, ndepths)
  ndepths = 1

  #-- open file
  f = Nio.open_file(path_to_output+"instance_0001/roms_posterior_0001_54939.nc","r")
  gridf = Nio.open_file(path_to_output+"instance_0001/useast_ini_0001.nc","r")
  lats=gridf.variables['lat_rho'][:,0]
  lons=gridf.variables['lon_rho'][0,:]
  # latitude lower and upper index
  lati_s = (np.abs(lats - latbound_s)).argmin()
  lati_n = (np.abs(lats - latbound_n)).argmin()
  loni_w = (np.abs(lons - lonbound_w)).argmin()
  loni_e = (np.abs(lons - lonbound_e)).argmin()
  lats_sub = lats[lati_s:lati_n]
  lons_sub = lons[loni_w:loni_e]
  #wks_type = "png"
  #wks = Ngl.open_wks(wks_type,"bc_pkc")
  #Ngl.define_colormap(wks,"default")         # Change color map.

  # Start date and end date
  start_date=datetime.datetime(2009,5,2)
  end_date=datetime.datetime(2009,8,31) # Fig 4.5-9


  #############################################################
  days_since=datetime.datetime(1858,11,17)
  start_date_ordinal=datetime.date.toordinal(start_date)
  end_date_ordinal=datetime.date.toordinal(end_date)
  days_since_ordinal=datetime.date.toordinal(days_since)
    
  start_date_index=start_date_ordinal-days_since_ordinal
  end_date_index=end_date_ordinal-days_since_ordinal
    
  # Optional sanity check
  #start_date_string=datetime.date.fromordinal(days_since_ordinal+start_date_index)
  nfiles=int((end_date_index-start_date_index+1)/2)
  filesi=list(range(start_date_index,end_date_index,2))
   
  #ffrac=2*((2*pi)/86400)*sin(26.010683 degrees)
  #dimsX = lons_sub.shape[0]
  #dimsY = lats_sub.shape[0]
  dimsX = lons.shape[0]
  dimsY = lats.shape[0]    
  h=gridf.variables['h'][:,:]
  Cs_r = gridf.variables['Cs_r']  # s_rho
  hc_nio   = gridf.variables['hc'] # single value
  hc = hc_nio.get_value()
  vtransform_nio = gridf.variables['Vtransform']
  vtransform = vtransform_nio.get_value()
  Sc_r = gridf.variables['s_rho'] # s_rho
  hinv=1/h[:,:]

  os.system("rm -rf ../ncs/mean_zeta_u_v_surface_full.nc")     #-- delete file if it exists
  dataset=Dataset('../ncs/mean_zeta_u_v_surface_full.nc','w',format='NETCDF4_CLASSIC')  #-- create dimensions
  lon = dataset.createDimension('lon', dimsX) # Want to compress what we have to look more like the figure; None ideally
  lat = dataset.createDimension('lat', dimsY) # Want to compress what we have to look more like the figure; None ideally
  nc_zeta = dataset.createVariable('zeta','f',('lat','lon'))
  nc_u = dataset.createVariable('u','f',('lat','lon'))
  nc_v = dataset.createVariable('v','f',('lat','lon'))

  t=f.variables['temp']
  #vin = t[0,:,lati_s:lati_n,loni_w:loni_e]
  vin = t[0,:,:,:]
  dims=vin.shape
  N=dims[0] 


        
  for t in range(0,nfiles):
    print("t=",t," nt=",nfiles)
    f = Nio.open_file(path_to_output+"analysis_mean."+str(filesi[t])+".nc","r")
    #z = f.variables['zeta'][0,lati_s:lati_n,loni_w:loni_e]
    #u = f.variables['u'][0,:,lati_s:lati_n,loni_w:loni_e-1]
    #v = f.variables['v'][0,:,lati_s:lati_n-1,loni_w:loni_e]
    z = f.variables['zeta'][0,:,:]
    u = f.variables['u'][0,:,:,:]
    v = f.variables['v'][0,:,:,:]

    njni=z.shape
    nj=njni[0]
    ni=njni[1]
    dimsu  = u.shape
    dimY   = dimsu[1]
    dimX   = dimsu[2]
    ur = np.empty(dims)

    ur[:,:,1:dimX-1] = 0.5*(u[:,:,:dimX-2] + u[:,:,1:dimX-1])
    ur[:,:,0]=ur[:,:,1]
    ur[:,:,dimX]=ur[:,:,dimX-1]
    
    dimsv  = v.shape
    dimY   = dimsv[1]
    dimX   = dimsv[2]

    vr = np.empty(dims)
    vr[:,1:dimY-1,:] = 0.5*(v[:,:dimY-2,:] + v[:,1:dimY-1,:])
    vr[:,0,:]=vr[:,1,:]
    vr[:,dimY,:]=vr[:,dimY-1,:]
    depth=np.empty(dims)
    uinterpolated=np.empty([ndepths,nj,ni])
    vinterpolated=np.empty([ndepths,nj,ni])  
    
    if vtransform == 2:
      for k in range(0,N):
        cff = 1/(hc + h[:,:])
        cffr = hc*Sc_r[k] + h[:,:]*Cs_r[k]
        depth[k,:,:]=(z + ( z + h )*cffr*cff)
    if vtransform == 1: 
      for k in range(0,N):
        cffr = hc*(Sc_r[k] - Cs_r[k])
        depth[k,:,:]=cffr+Cs_r[k]*h[:,:] + z*(1+(cffr+Cs_r[k]*h)*hinv)
        
    
    for i in range(0,ni-2):
      for j in range(0,nj-2):
        x=depth[:,j,i]
        uplane=ur[:,j,i]
        vplane=vr[:,j,i]
        ui=np.interp(depths,x,uplane)
        uinterpolated[:,j,i]=ui
        vi=np.interp(depths,x,vplane)
        vinterpolated[:,j,i]=vi    
    
    if t==0:
      zmean = z[:,:]
      umean = uinterpolated[:,:]
      vmean = vinterpolated[:,:]
    else:
      umean[:,:] = umean[:,:] + uinterpolated[:,:]
      vmean[:,:] = vmean[:,:] + vinterpolated[:,:]    
      zmean[:,:] = zmean[:,:] + z[:,:]
  zmean = zmean/nfiles
  umean = umean/nfiles
  vmean = vmean/nfiles

  nc_zeta[:,:] = zmean[:,:]
  nc_u[:,:] = umean[:,:]
  nc_v[:,:] = vmean[:,:]

  dataset.close()


t= 0  nt= 61
t= 1  nt= 61
t= 2  nt= 61
t= 3  nt= 61
t= 4  nt= 61
t= 5  nt= 61
t= 6  nt= 61
t= 7  nt= 61
t= 8  nt= 61
t= 9  nt= 61
t= 10  nt= 61
t= 11  nt= 61
t= 12  nt= 61
t= 13  nt= 61
t= 14  nt= 61
t= 15  nt= 61
t= 16  nt= 61
t= 17  nt= 61
t= 18  nt= 61
t= 19  nt= 61
t= 20  nt= 61
t= 21  nt= 61
t= 22  nt= 61
t= 23  nt= 61
t= 24  nt= 61
t= 25  nt= 61
t= 26  nt= 61
t= 27  nt= 61
t= 28  nt= 61
t= 29  nt= 61
t= 30  nt= 61
t= 31  nt= 61
t= 32  nt= 61
t= 33  nt= 61
t= 34  nt= 61
t= 35  nt= 61
t= 36  nt= 61
t= 37  nt= 61
t= 38  nt= 61
t= 39  nt= 61
t= 40  nt= 61
t= 41  nt= 61
t= 42  nt= 61
t= 43  nt= 61
t= 44  nt= 61
t= 45  nt= 61
t= 46  nt= 61
t= 47  nt= 61
t= 48  nt= 61
t= 49  nt= 61
t= 50  nt= 61
t= 51  nt= 61
t= 52  nt= 61
t= 53  nt= 61
t= 54  nt= 61
t= 55  nt= 61
t= 56  nt= 61
t= 57  nt= 61
t= 58  nt= 61
t= 59  nt= 61
t= 60  nt= 61
